In [1]:
from decouple import config

In [2]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [3]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [5]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key


api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [6]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (1min)'

In [10]:
results = data[dataset_key] # dict
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))

In [11]:
results

{'2024-11-15 19:59:00': {'1. open': '225.2501',
  '2. high': '225.2700',
  '3. low': '225.2500',
  '4. close': '225.2700',
  '5. volume': '452'},
 '2024-11-15 19:58:00': {'1. open': '225.2300',
  '2. high': '225.2600',
  '3. low': '225.2000',
  '4. close': '225.2600',
  '5. volume': '1028'},
 '2024-11-15 19:57:00': {'1. open': '225.1850',
  '2. high': '225.2500',
  '3. low': '225.1850',
  '4. close': '225.2200',
  '5. volume': '356'},
 '2024-11-15 19:56:00': {'1. open': '225.1850',
  '2. high': '225.2200',
  '3. low': '225.1500',
  '4. close': '225.1850',
  '5. volume': '428'},
 '2024-11-15 19:55:00': {'1. open': '225.1900',
  '2. high': '225.1900',
  '3. low': '225.1800',
  '4. close': '225.1800',
  '5. volume': '111'},
 '2024-11-15 19:54:00': {'1. open': '225.1900',
  '2. high': '225.2000',
  '3. low': '225.1512',
  '4. close': '225.1800',
  '5. volume': '1063'},
 '2024-11-15 19:53:00': {'1. open': '225.1700',
  '2. high': '225.1900',
  '3. low': '225.1700',
  '4. close': '225.1700',

In [12]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-11-15 19:59:00'

datetime.datetime(2024, 11, 6, 0, 59, tzinfo=<UTC>)

In [13]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '225.2501',
 '2. high': '225.2700',
 '3. low': '225.2500',
 '4. close': '225.2700',
 '5. volume': '452'}

In [14]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str,timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [15]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('225.2501'),
 'close_price': Decimal('225.2700'),
 'high_price': Decimal('225.2700'),
 'low_price': Decimal('225.2500'),
 'number_of_trades': None,
 'volume': 452,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 16, 0, 59, tzinfo=<UTC>)}

In [16]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('225.2501'), 'close_price': Decimal('225.2700'), 'high_price': Decimal('225.2700'), 'low_price': Decimal('225.2500'), 'number_of_trades': None, 'volume': 452, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 16, 0, 59, tzinfo=<UTC>)}
{'open_price': Decimal('225.2300'), 'close_price': Decimal('225.2600'), 'high_price': Decimal('225.2600'), 'low_price': Decimal('225.2000'), 'number_of_trades': None, 'volume': 1028, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 16, 0, 58, tzinfo=<UTC>)}
{'open_price': Decimal('225.1850'), 'close_price': Decimal('225.2200'), 'high_price': Decimal('225.2500'), 'low_price': Decimal('225.1850'), 'number_of_trades': None, 'volume': 356, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 16, 0, 57, tzinfo=<UTC>)}
{'open_price': Decimal('225.1850'), 'close_price': Decimal('225.1850'), 'high_price': Decimal('225.2200'), 'low_price': Decimal('225.1500'), 'number_of_trades': None, '

In [17]:
import pandas as pd

# Load the JSON data into a pandas DataFrame
df = pd.json_normalize(new_data)

# Display the DataFrame
display(df)

,open_price,close_price,high_price,low_price,number_of_trades,volume,volume_weighted_average,time
0,225.0400,225.1100,225.1300,225.0400,None,503,None,2024-11-15 23:20:00+00:00
